<a href="https://colab.research.google.com/github/nehach29/Flask-API-and-Streamlit-of-Detectron2-model/blob/main/Streamlit_Mask_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><b>Streamlit API of Mask Detection Using Detecton - 2<br>

<br> Project Group : 4 <br>
<br> By:<br>
Neha Chaudhary <br>
Yash Kumar

<b><h1> Installing and loading necessary modules

In [ ]:
!git clone https://github.com/facebookresearch/detectron2.git

In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
!pip install -q streamlit

In [ ]:
!npm install localtunnel

In [ ]:
!git clone https://github.com/IYashCanCode/Detectron2-Mask-Detection.git

Cloning into 'Detectron2-Mask-Detection'...
remote: Enumerating objects: 441, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 441 (delta 0), reused 0 (delta 0), pack-reused 439
Receiving objects: 100% (441/441), 89.32 MiB | 25.46 MiB/s, done.
Resolving deltas: 100% (27/27), done.


In [ ]:
import os
os.mkdir('/content/Detectron2-Mask-Detection/uploads')

In [ ]:
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.structures import BoxMode
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.logger import setup_logger
setup_logger()

import os
import cv2
import pickle
import random
from PIL import Image
import streamlit as st
import matplotlib.pyplot as plt
from flask import Flask,request,render_template,jsonify,send_file
from google.colab.patches import cv2_imshow

<h3><b>Registering Training images coco format

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("OD_Mask", {}, "/content/Detectron2-Mask-Detection/sample_train/train_.json", "/content/Detectron2-Mask-Detection/sample_train/images")

In [ ]:
OD_Mask_metadata = MetadataCatalog.get("OD_Mask")
dataset_dicts = DatasetCatalog.get("OD_Mask")

WARNING [12/29 08:11:43 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/29 08:11:43 d2.data.datasets.coco]: Loaded 337 images in COCO format from /content/Detectron2-Mask-Detection/sample_train/train_.json


In [ ]:
OD_Mask_metadata

namespace(name='OD_Mask',
          json_file='/content/Detectron2-Mask-Detection/sample_train/train_.json',
          image_root='/content/Detectron2-Mask-Detection/sample_train/images',
          evaluator_type='coco',
          thing_classes=['Mask', 'No Mask'],
          thing_dataset_id_to_contiguous_id={0: 0, 1: 1})

<h3><b>Training Detectron2 detection model on custome dataset of mask detection

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_R_50_C4_1x.yaml'))
cfg.DATASETS.TRAIN = ("OD_Mask",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-Detection/faster_rcnn_R_50_C4_1x.yaml')
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.02
cfg.MODEL.DEVICE='cuda'
cfg.SOLVER.MAX_ITER = 250
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[12/29 08:11:45 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
 

model_final_721ade.pkl: 136MB [00:01, 130MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[12/29 08:11:46 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[12/29 08:12:06 d2.utils.events]:  eta: 0:00:16  iter: 19  total_loss: 1.293  loss_cls: 0.4803  loss_box_reg: 0.5785  loss_rpn_cls: 0.1309  loss_rpn_loc: 0.01393    time: 0.5643  last_time: 0.5455  data_time: 0.0318  last_data_time: 0.0325   lr: 0.0076124  max_mem: 2104M
[12/29 08:12:20 d2.utils.events]:  eta: 0:00:05  iter: 39  total_loss: 0.8053  loss_cls: 0.2051  loss_box_reg: 0.4634  loss_rpn_cls: 0.04208  loss_rpn_loc: 0.01171    time: 0.5609  last_time: 0.5366  data_time: 0.0143  last_data_time: 0.0072   lr: 0.015604  max_mem: 2104M
[12/29 08:12:26 d2.utils.events]:  eta: 0:00:00  iter: 49  total_loss: 0.7569  loss_cls: 0.2019  loss_box_reg: 0.4611  loss_rpn_cls: 0.05232  loss_rpn_loc: 0.01387    time: 0.5565  last_time: 0.5122  data_time: 0.0144  last_data_time: 0.0059   lr: 0.0196  max_mem: 2104M
[12/29 08:12:26 d2.engine.hooks]: Overall training speed: 48 iterations in 0:00:26 (0.5566 s / it)
[12/29 08:12:26 d2.engine.hooks]: Total training time: 0:00:30 (0:00:03 on hooks)


<h2><b>Streamlit API for custom mask detection detectron2 model build<br><br><h5><i><b>Please use the link and enter the number sepearted by four ' . '(dots)  in the new opened tab to continue. <br>Example: 34.0.123.177

In [ ]:
%%writefile app2.py

from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.structures import BoxMode
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.logger import setup_logger
setup_logger()

import os
import cv2
import pickle
import random
from PIL import Image
import streamlit as st
import matplotlib.pyplot as plt
from flask import Flask,request,render_template,jsonify,send_file
from google.colab.patches import cv2_imshow






setup_logger()
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_R_50_C4_1x.yaml'))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-Detection/faster_rcnn_R_50_C4_1x.yaml')
cfg.MODEL.DEVICE='cuda'
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR,'model_final.pth')
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75
predictor = DefaultPredictor(cfg)

st.title("Detectron2 Mask Detection Web App")


uploaded_image = st.file_uploader("Choose an image...", type="jpg")



if uploaded_image is not None:
    image = Image.open(uploaded_image)
    st.image(image, caption="Uploaded Image.", use_column_width=True)
    image.save('./Detectron2-Mask-Detection/uploads/image1.jpg')

    # Add a "Predict" button
    if st.button("Predict"):
        im = cv2.imread('./Detectron2-Mask-Detection/uploads/image1.jpg')
        outputs = predictor(im)
        v = Visualizer(im[:, :, ::-1],
                      metadata={'thing_classes':['Mask', 'No Mask']},
                      scale=1.,
                      instance_mode=ColorMode.IMAGE
                      )
        v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        v.save('./Detectron2-Mask-Detection/uploads/annotated_images_1.jpg')
        st.image('./Detectron2-Mask-Detection/uploads/annotated_images_1.jpg', use_column_width=True)

Overwriting app2.py


In [ ]:
!streamlit run app2.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.32.173.219
npx: installed 22 in 2.769s
your url is: https://empty-crabs-deny.loca.lt
